In [1]:
%load_ext memory_profiler

In [2]:
import yaml
import tensorflow as tf
from tensorflow_tts.configs import LightSpeechConfig, MultiBandMelGANGeneratorConfig
from tensorflow_tts.models import TFLightSpeech, TFMelGANGenerator

/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
def infer(m, l):
    with open(m) as f:
        melgan_config = yaml.load(f, Loader=yaml.Loader)

    with open(l) as f:
        lightspeech_config = yaml.load(f, Loader=yaml.Loader)
    
    lightspeech = TFLightSpeech(config=LightSpeechConfig(**lightspeech_config["lightspeech_params"]))
    generator = TFMelGANGenerator(MultiBandMelGANGeneratorConfig(**melgan_config["multiband_melgan_generator_params"]))

    input_ids = tf.convert_to_tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]], tf.int32)
    speaker_ids = tf.convert_to_tensor([0], tf.int32)
    duration_gts = tf.convert_to_tensor([[50, 50, 50, 50, 50, 50, 50, 50, 50, 50]], tf.int32)
    f0_gts = tf.convert_to_tensor([[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]], tf.float32)
    energy_gts = tf.convert_to_tensor([[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]], tf.float32)

    mel, *_ = lightspeech(input_ids, speaker_ids, duration_gts, f0_gts, energy_gts)
    _ = generator(mel)

In [9]:
%memit infer("../examples/multiband_melgan_hf/conf/multiband_melgan_hf.en.mini.yml", "../examples/lightspeech/conf/lightspeech_englishipa_small.yaml")

peak memory: 820.69 MiB, increment: 69.96 MiB


In [5]:
%memit infer("../examples/multiband_melgan_hf/conf/multiband_melgan_hf.en.v1.yml", "../examples/lightspeech/conf/lightspeech_englishipa.yaml")

peak memory: 864.38 MiB, increment: 121.29 MiB
